# CH16 코루틴 
* `yield` 키워드 -> 생산하다 / 양보하다
* `next()` 와 `send()` 메서드로 제너레이터 실행 제어

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?
* 코루틴 : 제너레이터가 호출자에 데이터를 생성해주고 호출자로부터 데이터를 받으면서 호출자와 협업하는 프로시저
    * `send()` : 제너레이터 내부의 yield 표현식의 값이 될 데이터 전송
    * `throw()` : 제너레이터 내부에서 처리할 예외를 호출자가 발생시켜줌 
    * `close()` : 제너레이터를 종료시킴

## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

In [1]:
def simple_coroutine():
    print('-> corotine started')
    x = yield  # 값을 생성 안할 수도 있음
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7f5f79ae4e50>

In [4]:
next(my_coro)

-> corotine started


In [5]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

*** inspect.getgeneratorstate() 의 상태 구분
* 'GEN_CREATED'
* 'GEN_RUNNING'
* 'GEN_SUSPENDED'
* 'GEN_CLOSED'

In [6]:
my_coro = simple_coroutine()
my_coro.send(1729)  # 코루틴을 기동(priming)하기 위해선, next() 가 무조건 처음 호출되어야 함

TypeError: can't send non-None value to a just-started generator

In [7]:
def simple_coro2(a):  # yield 를 두번 이상 호출하는 예제
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [8]:
my_coro2 = simple_coro2(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [9]:
next(my_coro2)

-> Started: a = 14


14

In [10]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [11]:
my_coro2.send(28)

-> Received: b = 28


42

In [12]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [13]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

## 16.3 예제: 이동 평균을 계산하는 코루틴

In [76]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average  # 처음 next() 에는 None 를 리턴
        total += term
        count += 1
        average = total/count

In [78]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [16]:
coro_avg.send(30)

20.0

In [17]:
coro_avg.send(5)

15.0

## 16.4 코루틴을 기동하기 위한 데커레이터
* 코루틴이 생성될 때, `next()` 를 자동으로 실행해주는 데커레이터

In [18]:
from functools import wraps

def coroutine(func):
    """데커레이터: `func`를 기동해서 첫 번째 `yield`까지 진행한다."""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [81]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [20]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [21]:
coro_avg.send(10)

10.0

In [22]:
coro_avg.send(30)

20.0

In [23]:
coro_avg.send(5)

15.0

*** yield from 구문과는 사용 불가

## 16.5 코루틴 종료와 예외 처리
* 코루틴 안에서 발생한 예외를 처리하지 않으면, `next()`나 `send()`로 호출한 호출자에 예외 전파

In [86]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [87]:
coro_avg.send(50)

45.0

In [88]:
coro_avg.send('spam')  # 코루틴 안에서 예외를 처리하지 않았으므로, 코루틴 종료

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [89]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

In [90]:
coro_avg.send(60)

StopIteration: 

In [100]:
# 예외를 SEND 하는 방법?
coro_avg = averager()
coro_avg.send(StopIteration) ## ???
#coro_avg.throw(StopIteration)

TypeError: unsupported operand type(s) for +=: 'float' and 'type'

In [101]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

In [102]:
class DemoException(Exception):
    """설명에 사용할 예외 유형"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoExeceptionhandled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')  # while 문 빠져나오면 바로 코루틴 종료되므로 실행 안됨


In [103]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [104]:
exc_coro.send(11)

-> coroutine received: 11


In [105]:
exc_coro.send(22)

-> coroutine received: 22


In [106]:
exc_coro.close()

In [107]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [ ]:
# 처리된 예외를 던지면?? 종료되지 않음

In [108]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [109]:
exc_coro.send(11)

-> coroutine received: 11


In [110]:
exc_coro.throw(DemoException)

*** DemoExeceptionhandled. Continuing...


In [111]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [112]:
exc_coro.send(40)

-> coroutine received: 40


In [41]:
# 처리되지 않은 예외를 던지면? 종료

In [113]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [114]:
exc_coro.send(11)

-> coroutine received: 11


In [115]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [116]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [ ]:
# 종료될 때, 로직이 필요한 경우 finally 구문을 이용

In [137]:
class DemoException(Exception):
    """설명에 사용할 예외 유형"""
    
def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoExeceptionhandled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [138]:
exc_coro = demo_finally()
next(exc_coro)

-> coroutine started


In [139]:
exc_coro.send(11)

-> coroutine received: 11


In [140]:
exc_coro.close()
# exc_coro.throw(ZeroDivisionError)
# exc_coro.throw(DemoException)

-> coroutine ending


In [141]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

## 16.6 코루틴에서 값 반환하기
* 기존 제너레이터는 return 이 불가능했지만, 코루틴 지원 이후 가능해짐

In [63]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [64]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [ ]:
# 예외가 전파되어 종료된 경우, 예쁘게 반환 되지는 않음

In [65]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.5)

In [ ]:
## 너무 짜친거 같기는 한데, yield from 에서도 같은 방법으로 StopIteration 예외 처리하여 반환

## 16.7 yield from 사용하기
* 다른 언어에서 비슷한 역할을 하는 구문 : `await`
* 제너레이터 `gen()` 이 `yield from subgen()` 을 호출하고, `subgen()`이 이어받아 값을 생성하고 `gen()` 호출자에 반환
    * 실질적으로 `subgen()`이 호출자랑 협업
    * `gen()`은 `subgen()`이 종료될 때까지 실행 중단

In [66]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

In [67]:
list(gen())

['A', 'B', 1, 2]

In [68]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

In [69]:
def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

** for 루프를 간결하게 대체하는데에도 쓰이지만, 중첩 제너레이터를 복잡하게 사용할때 진정한 의미가 있음

* `yield from` 의 주요한 특징은 가장 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다.
    * 이 둘이 직접 값을 주고 받으며, 중간에 있는 코루틴이 예외 처리 코드를 구현할 필요 없이 예외를 직접 던질 수 있다. => 코루틴 위임 (coroutine delegation)



* 대표 제너레이터 (delegating generator) : `yield from <반복형>` 을 포함하는 제너레이터 함수
* 하위 제너레이터 (subgenerator) : `<반복형>` 에서 가져오는 제너레이터
* 호출자 (caller) : 대표 제너레이터를 호출하는 코드, `클라이언트`라고 쓰이기도 함.


In [72]:
!python coroaverager3.py

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [73]:
# debug
!python coroaverager3.py

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [74]:
# debug (send None 제거했을때)
!python coroaverager3.py
## 하위 제너레이터 함수의 리턴을 못받음 (코루틴이 제 타이밍에 종료가 되지 않아서.. 가비지 컬렉트에 의해 종료)


{}


![그림16-2](pic16-2.png)

## 16.8 yield from 의 의미
PEP 380 내용
* 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자(즉, 클라이언트)에 바로 전달된다.
* `send()` 를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 None 이면 하위 제너레이터의 `__next__()` 메서드가 호출된다. 전달된 값이 None 이 아니면 하위 제너레이터의 `send()` 메서드가 호출된다. 호출된 메서드에서 `StopIteration` 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
* 제너레이터나 하위 제너레이터에서 `return expr` 문을 실행하면, 제너레이터를 빠져나온 후 StopIteraton(expr) 예외가 발생한다.
* 하위 제너레이터가 실행을 마친 후 발생한 `StopIteration` 예외의 첫 번째 인수가 `yield from` 표현식의 값이 된다.
* 대표 제너레이터에 던져진 `GeneratorExit` 이외의 예외는 하위 제너레이터의 `throw()` 메서드에 전달된다. `throw()` 메서드를 호출해서 `StopIteration` 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그외의 예외는 대표 제너레이터에 전달된다.
* `GeneratorExit` 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 `close()` 메서드가 호출되면 하위 제너레이터에 `close()` 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 대표 제너레이터에서 `GeneratorExit` 예외가 발생한다. => ?????????

`RESULT = yield from EXPR`

위에 예제코드에서의 기능은 아래의 코드

In [ ]:
_i = iter(EXPR)
try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        _s = yield _y
        try:
            _y = _i.send(_s)
        except StopIteration as _e:
            _r = _e.value
            break
Result = _r

`RESULT = yield from EXPR`

실제로 동작하는 코드

In [ ]:
_i = iter(EXPR)
try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        try:
            _s = yield _y
        except GeneratorExit as _e:
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        except BaseException as _e:
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:
            try:
                if _s is None:
                    _y = next(_i)
                else:
                    _y = _i.send(_s)
            except StopIteration as _e:
                _r = _e.value
                break
Result = _r

## 16.9 사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴
### 16.9.1 이산 이벤트 시뮬레이션에 대해
* 이산 이벤트 시뮬레이션 (Discrete Event Simulation)

### 16.9.2 택시 집단 시뮬레이션

In [143]:
!python taxi_sim.py -s 3

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=8, proc=1, action='pick up passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 2        Event(time=15, proc=2, action='pick up passenger')
taxi: 2        Event(time=17, proc=2, action='drop off passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 2        Event(time=18, proc=2, action='pick up passenger')
taxi: 2        Event(time=25, proc=2, action='drop off passenger')
taxi: 1     Event(time=27, proc=1, action='drop off passenger')
taxi: 2        Event(time=27, proc=2, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='pick up passenger')
taxi: 2        Event(time=40, proc=2, action='drop off passenger')
taxi: 2        Event(time=44, proc=2, action='pick up passenger')
taxi: 1     Event(time=55, proc=1, action='pick up passen

In [ ]:
어렵..